In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from scipy.stats import pearsonr
import statsmodels.api as sm
import geopandas as gpd
from scipy.stats import f_oneway

In [ ]:
df_GR = pd.read_csv('GLaridHumid_95.csv')
def calculate_row_means(df1,df2):
    df_concat = pd.merge(df1,df2, on = 'site')
    df_concat.loc[:,'1901':].mean(axis= 1)
    return df_concat.loc[:,'1901':].mean(axis= 1).to_list()
df_species = pd.read_excel('40tree.xlsx')
species_dict = dict(zip(df_species['Tree_Speci'], df_species['Clade']))
df_GR['Species_group'] = df_GR['Tree_Speci'].map(species_dict)
df_GR['Species_group'] = df_GR['Species_group'].fillna('Unknown')
df_GR['log_aiv3']= np.log10(df_GR['ai_v3'])
print("含有 -inf 的行数: ", np.isinf(df_GR['log_aiv3']).sum())
df_GR['log_aiv3'].replace(-np.inf, np.nan, inplace=True)
df_GR.dropna(subset=['log_aiv3'], inplace=True)
x = df_GR[df_GR['Species_group']=='Gymnosperms']['log_aiv3']
y = df_GR[df_GR['Species_group']=='Gymnosperms']['Growth_loss']
r, p = stats.pearsonr(x, y)
print(f'r:{r}')
print(f'p:{p}')
x = df_GR[df_GR['Species_group']=='Angiosperms']['log_aiv3']
y = df_GR[df_GR['Species_group']=='Angiosperms']['Growth_loss']
r, p = stats.pearsonr(x, y)
print(f'r:{r}')
print(f'p:{p}')

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)

species_groups = df_GR['Species_group'].unique()


color_map = {
    species_groups[0]: (0/255, 92/255, 230/255),
    species_groups[1]: (255/255, 0/255, 0/255)
}

for species in species_groups:
    df_group = df_GR[df_GR['Species_group'] == species]

    r, p = pearsonr(df_group['log_aiv3'], df_group['Growth_loss'])

    X = df_group[['log_aiv3']]
    y = df_group['Growth_loss']

    X_with_const = sm.add_constant(X)  
    model = sm.OLS(y, X_with_const).fit()

    intercept, slope = model.params

    predictions = model.get_prediction(X_with_const)
    predicted_mean = predictions.predicted_mean
    conf_int = predictions.conf_int()

    plt.scatter(df_group['log_aiv3'], df_group['Growth_loss'], 
                alpha=0.06, label=f'{species} (r={r:.2f}, p={p:.2e})',
                color=color_map[species])

    plt.plot(df_group['log_aiv3'], predicted_mean, color=color_map[species], linestyle='-')

    plt.fill_between(df_group['log_aiv3'], conf_int[:, 0], conf_int[:, 1], color=color_map[species], alpha=0.2)

plt.yticks([-0.4, 0, 0.5])
plt.ylim(-0.4,0.7)
plt.xlim(-1,1)
plt.xlabel('Log10(Aridity index)')
plt.ylabel('Growth loss',fontsize=18)
plt.legend(fontsize=10,loc=1)
plt.grid(True)
plt.savefig('Fig2d.jpg', dpi=300, bbox_inches='tight')
plt.show()
shapefile_path = r"E:\experiment\DTR_AI_growth\code4\EcoReClass\Ecoregion2017ReCalss.shp"
gdf = gpd.read_file(shapefile_path)
geometry = gpd.points_from_xy(df_GR.Longitude, df_GR.Latitude)
geo_df_GR = gpd.GeoDataFrame(df_GR, geometry=geometry)
geo_df_GR.set_crs(gdf.crs, inplace=True)
joined = gpd.sjoin(geo_df_GR, gdf, how="left", op="within")
df_GR['Biome_Re'] = joined['Biome_Re']
df_GR = df_GR.dropna()
df_GR.to_csv('gl_Biome.csv',index=False)
df_GR['Biome_Re'].unique()
df_GRdrop  = df_GR.dropna(subset=['Biome_Re'])

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(12, 6))
alpha = 0.7 
colors = [(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
sns.boxplot(x='Biome_Re', y='Growth_loss', hue='AI_Class', data=df_GRdrop, palette=colors
           ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
biomes = df_GRdrop['Biome_Re'].unique()
for biome in biomes:
    if biome == 'Deserts':
        continue  
    
    dry = df_GRdrop[(df_GRdrop['Biome_Re'] == biome) & (df_GRdrop['AI_Class'] == 'Arid')]['Growth_loss']
    wet = df_GRdrop[(df_GRdrop['Biome_Re'] == biome) & (df_GRdrop['AI_Class'] == 'Humid')]['Growth_loss']

    t_stat, p_val = stats.ttest_ind(dry, wet, nan_policy='omit')
    print(biome)
    print(t_stat)
    print(p_val)

    x1, x2 = biomes.tolist().index(biome) - 0.2, biomes.tolist().index(biome) + 0.2
    y, h, col = 0.52, 0.02, 'k'  
    if np.isfinite(y):
        plt.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, c=col)
        significance = ''
        if p_val < 0.001:
            significance = '***'
        elif p_val < 0.01:
            significance = '**'
        elif p_val < 0.05:
            significance = '*'
        
        plt.text((x1 + x2) * .5, y + h, f'{significance}', ha='center', va='bottom', color=col)
    n_dry = len(dry)
    n_wet = len(wet)
    plt.text(biomes.tolist().index(biome) + 0.25, y - 0.05, f'n = {n_dry}', ha='center', va='center', color='red')
    plt.text(biomes.tolist().index(biome) - 0.25, y - 0.05, f'n = {n_wet}', ha='center', va='center', color='blue')
plt.xticks(rotation=45)
plt.xlabel('')
plt.ylabel('Growth loss',fontsize=18)
plt.legend(title='Region')
plt.savefig('FigS4d.jpg', dpi=300, bbox_inches='tight')
plt.show()
grouped = df_GRdrop.groupby(['Biome_Re', 'AI_Class']).apply(lambda x: pd.Series({
    'Growth_loss>0': (x['Growth_loss'] > 0).mean() * 100,
    'Growth_loss<=0': (x['Growth_loss'] <= 0).mean() * 100
})).reset_index()
grouped.to_csv('Biome_GL.csv')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
fig, ax = plt.subplots(figsize=(6, 6))
biomes = grouped['Biome_Re'].unique()
width = 0.35  
colors = [(255/255, 0/255, 0/255),(0/255, 92/255, 230/255)]
for i, biome in enumerate(biomes):
    subset = grouped[grouped['Biome_Re'] == biome]
    arid_growth_loss_gt_0 = subset[subset['AI_Class'] == 'Arid']['Growth_loss>0'].values[0] if not subset[subset['AI_Class'] == 'Arid'].empty else 0
    arid_growth_loss_le_0 = subset[subset['AI_Class'] == 'Arid']['Growth_loss<=0'].values[0] if not subset[subset['AI_Class'] == 'Arid'].empty else 0
    humid_growth_loss_gt_0 = subset[subset['AI_Class'] == 'Humid']['Growth_loss>0'].values[0] if not subset[subset['AI_Class'] == 'Humid'].empty else 0
    humid_growth_loss_le_0 = subset[subset['AI_Class'] == 'Humid']['Growth_loss<=0'].values[0] if not subset[subset['AI_Class'] == 'Humid'].empty else 0
    
    ax.bar(biome, arid_growth_loss_gt_0, width, color=colors[0], label='Arid Growth_loss>0' if i == 0 else "",alpha=0.7)
    ax.bar(biome, arid_growth_loss_le_0, width, bottom=arid_growth_loss_gt_0, color=colors[1], label='Arid Growth_loss<=0' if i == 0 else "",alpha=0.7)
    ax.bar(biome, humid_growth_loss_gt_0, width, bottom=arid_growth_loss_gt_0 + arid_growth_loss_le_0, color=colors[0], hatch='//', label='Humid Growth_loss>0' if i == 0 else "",alpha=0.7)
    ax.bar(biome, humid_growth_loss_le_0, width, bottom=arid_growth_loss_gt_0 + arid_growth_loss_le_0 + humid_growth_loss_gt_0, color=colors[1], hatch='//', label='Humid Growth_loss<=0' if i == 0 else "",alpha=0.7)

ax.set_ylabel('Percentage(%)',fontsize=18)
ax.set_xticks(range(len(biomes)))
ax.set_xticklabels(biomes, rotation=45)
plt.tight_layout()
plt.savefig('Fig2b.jpg', dpi=600, bbox_inches='tight')
plt.show()
result_list = []
grouped = df_GRdrop.groupby('Tree_Speci')

for name, group in grouped:
    positive_count = (group['Growth_loss'] > 0).sum()
    negative_count = (group['Growth_loss'] < 0).sum()
    total_count = len(group)
    
    proportion_positive = (positive_count / total_count) * 100
    proportion_negative = (negative_count / total_count) * 100
    result_list.append({
        'Tree_Speci': name,
        'Proportion_Growth_Loss_Positive': proportion_positive,
        'Proportion_Growth_Loss_Negative': proportion_negative
    })

result_df = pd.DataFrame(result_list)
result_df.to_csv('TreespeciesGL.csv')
grouped = df_GRdrop.groupby(['Species_group', 'AI_Class']).apply(lambda x: pd.Series({
    'Growth_loss>0': (x['Growth_loss'] > 0).mean() * 100,
    'Growth_loss<=0': (x['Growth_loss'] <= 0).mean() * 100
})).reset_index()
grouped.to_csv('Species_group.csv')
Gymnosperms = df_GRdrop[df_GRdrop['Species_group']=='Gymnosperms']
Angiosperms = df_GRdrop[df_GRdrop['Species_group']=='Angiosperms']
f_statistic_dr, p_value_dr = f_oneway(Gymnosperms['Growth_loss'], Angiosperms['Growth_loss'])
print(p_value_dr)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
colors = [(255/255, 0/255, 0/255),(0/255, 92/255, 230/255)]
plt.figure(figsize=(4,3.5),dpi=100)
#palette=['#8DCDD5','#E6846D']
alpha = 0.5 
sns.boxplot(x="Species_group", y="Growth_loss", data=df_GRdrop
            ,palette=colors,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.ylim(-0.3,0.6)
plt.yticks([-0.3,0,0.3,0.6]) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('Growth loss',fontsize=18)
plt.text(0.5, 0.5, '***', ha='center', va='center')
plt.text('Gymnosperms', 0.5,  f'n= {len(Gymnosperms)}', ha='center', va='center')
plt.text('Angiosperms', 0.5,  f'n= {len(Angiosperms)}', ha='center', va='center')
plt.savefig('FigS4a.jpg',dpi=300, bbox_inches='tight')
plt.show()